In [1]:
import os
import sys
import time
import copy
import cPickle
import random

import numpy as np
import scipy
import scipy.sparse as sp
from collections import Counter
import matplotlib.pyplot as plt
from tsne import bh_sne
import mpld3
import matplotlib.cm as cm
import matplotlib.patches as mpatches

sys.path.append("/Users/corbinrosset/Dropbox/Arora/QA-code/src/")

import KBC_Text




In [2]:
datapath = '/Users/corbinrosset/Dropbox/Arora/QA-code/src/KBC_Text/data/'
dataset = 'FB15k'
relation_key_words = set(['film', 'government', 'tv', 'location', 'organization', \
                          'education', 'award', 'football', 'people', 'olympics', \
                          'music', 'military', 'popstra', 'business', 'baseball', 'NotFound'])
entity_key_words = set(['film', 'university', 'award', 'county', 'football', 'team', 'city', 'national', 'actor', \
 'congress', 'england', 'history', 'performance', 'california', 'war', 'america', \
  'Party', 'Basketball', 'tv', 'france', 'King', 'NotFound'])

# set(['film', 'sports', 'government', 'tv', 'location', 'organization', 'education', \
#  'award', 'football', 'people', 'olympics', 'team', 'music', \
#  'military', 'popstra', 'business', 'baseball', \
#  'player', 'position', 'fictional_universe', \
#  'basketball', 'measurement_unit', 'soccer', 'currency', 'person', \
#  'religion', 'medicine', 'computer', 'ice_hockey', 'money', 'book', \
#  'travel', 'aviation', 'broadcast', 'country', 'actor', 'roster', 'NotFound'])

def load_file(path):
    return scipy.sparse.csr_matrix(cPickle.load(open(path)),
            dtype=np.float64)


def convert2idx(spmat):
    rows, cols = spmat.nonzero()
    return rows[np.argsort(cols)]


def load_FB15k_data():
    # Positives
    trainl = load_file(datapath + dataset + '-train-lhs.pkl') # rows = entities, col are one hot to indicate which is left entity
    trainr = load_file(datapath + dataset + '-train-rhs.pkl') # rows = relations, but only in bottom 1,345 rows
    traino = load_file(datapath + dataset + '-train-rel.pkl')

#     print trainl.tocoo()
    print traino.tocsc()

    
    # Valid set
    validl = load_file(datapath + dataset + '-valid-lhs.pkl')
    validr = load_file(datapath + dataset + '-valid-rhs.pkl')
    valido = load_file(datapath + dataset + '-valid-rel.pkl')
#     print np.shape(validl)


    # Test set
    testl = load_file(datapath + dataset + '-test-lhs.pkl')
    testr = load_file(datapath + dataset + '-test-rhs.pkl')
    testo = load_file(datapath + dataset + '-test-rel.pkl')
    

    # create list of which rows were hot for each of the 483k columns (each column is one hot)
    trainlidx = convert2idx(trainl) # extract the first neval left entity examples (just take which row is hot in trainl)
    trainridx = convert2idx(trainr)
    trainoidx = convert2idx(traino) # take the rows that are hot in the first neval columns of traino
    validlidx = convert2idx(validl)
    validridx = convert2idx(validr)
    validoidx = convert2idx(valido)
    testlidx = convert2idx(testl)
    testridx = convert2idx(testr)
    testoidx = convert2idx(testo)
    print np.shape(trainlidx)
    print np.shape(trainridx)
    print np.shape(trainoidx)
    
    true_triples=np.concatenate([testlidx,validlidx,trainlidx,testoidx,validoidx,trainoidx,testridx,validridx,trainridx]).reshape(3,testlidx.shape[0]+validlidx.shape[0]+trainlidx.shape[0]).T
    KB = set([tuple(i) for i in true_triples]) ### {(head, rel, tail)}
    assert len(KB) == np.shape(true_triples)[0]
    
    return trainlidx, trainridx, trainoidx, validlidx, validridx, validoidx, testlidx, testridx, testoidx, true_triples, KB
        
def load_embeddings(path):
    print'loading model from file: ' + str(path)
    embeddings = cPickle.load(open(path))
    ### TODO: write the word embeddings to file as well
    entityEmbs, relEmbsL, relEmbsR = embeddings[0].E.get_value().T, embeddings[2].E.get_value().T, embeddings[2].E.get_value().T
    return entityEmbs, relEmbsL, relEmbsR

def t_sne(data):
    '''run_bh_tsne(data, no_dims=2, perplexity=50, theta=0.5, randseed=-1, verbose=False,initial_dims=50, use_pca=True, max_iter=1000):
        
        Run TSNE based on the Barnes-HT algorithm

        Parameters:
        ----------
        data: file or numpy.array
            The data used to run TSNE, one sample per row
        no_dims: int
        perplexity: int
        randseed: int
        theta: float
        initial_dims: int
        verbose: boolean
        use_pca: boolean
        max_iter: int
    '''

    X_2d = bh_sne(data, perplexity=40, theta=0.5) #, max_iter=600) #bh_sne(X, perplexity=50, theta=0.5)  
    return X_2d

def tooltip_style(text):
    return "<p style=\"color: #ffffff; background-color: #000000\">" + str(text) + "</p>"

def get_label_rel(relation):
    for i in relation_key_words:
        if i in str(relation):
            return i
    return 'NotFound'

def get_label_ent(entity):
    entity_key_words
    for i in entity_key_words:
        if i in str(entity.lower()):
            return i
    return 'NotFound'
    
    
def plot_embeddings(X, Y, popup_labels, title):
    '''X is a list of matrices of length n, each of which has its own class or 
    color, y. popup_labels is also a list of n lists, each containing the tag
    of individual points (rows in a matrix of X)'''

    ###############
    labels_set = set(entity_key_words).copy() ### change THIS for relation/entities
    labels_set.remove('NotFound')
    
    legend_partition, legend_strings = [], [] 
    l_keys, l_nums, recs = [], [], []
    fig, ax = plt.subplots(subplot_kw=dict(axisbg='#EEEEEE'), figsize=(20,20))
    
    assert len(X) == len(Y)
    if popup_labels:
        assert len(popup_labels) == len(X)
        assert np.shape(X)[0] == np.shape(popup_labels)[0]
    ax.grid(color='white', linestyle='solid')
    colors = cm.rainbow(np.linspace(0, 1, len(labels_set)))
    color_label_map = {}
    for c, label in zip(colors, labels_set):
        color_label_map[label] = c
        legend_strings.append(label)
        
    for (x, group_label, tag) in zip(X, Y, popup_labels):
        # label = get_label_rel(tag) # Change THIS
        label = get_label_ent(tag)
        if label == 'NotFound':
            continue
        scatter = ax.scatter(x[0], x[1], c = color_label_map[label], alpha=1.0, label=label)
        legend_partition.append(scatter)
        tooltip = mpld3.plugins.PointHTMLTooltip(scatter, labels=[tooltip_style(tag)])
        mpld3.plugins.connect(fig, tooltip)
        
    print 'Legend: '
    for i, (key, value) in enumerate(color_label_map.items()):
        print str(i) + ' = ' + str(key)
        l_nums.append(str(i))
        l_keys.append(key)
        recs.append(mpatches.Rectangle((0,0),1,1,fc=value))

    plt.legend(recs, l_keys, loc='upper right', fontsize=8)
    if title:
        ax.set_title(title, size=12)
        
    mpld3.save_html(fig, title + '.html')
    mpld3.show()
#     mpld3.display() # doesn't seem to work for ipython notebooks?
    return

def plot_embedding_pairs(X1, X2, Y, popup_labels, title):
    '''X is a list of matrices of length n, each of which has its own class or 
    color, y. popup_labels is also a list of n lists, each containing the tag
    of individual points (rows in a matrix of X)'''

    ###############
    labels_set = set(relation_key_words).copy() ### change THIS for relation/entities
    labels_set.remove('NotFound')
    
    legend_partition, legend_strings = [], [] 
    l_keys, l_nums, recs = [], [], []
    fig, ax = plt.subplots(subplot_kw=dict(axisbg='#EEEEEE'), figsize=(20,20))
    
    assert len(X) == len(Y)
    if popup_labels:
        assert len(popup_labels) == len(X)
        assert np.shape(X)[0] == np.shape(popup_labels)[0]
    ax.grid(color='white', linestyle='solid')
    colors = cm.rainbow(np.linspace(0, 1, len(labels_set)))
    color_label_map = {}
    for c, label in zip(colors, labels_set):
        color_label_map[label] = c
        legend_strings.append(label)
        
    for (x1, x2, group_label, tag) in zip(X1, X2, Y, popup_labels):
        label = get_label_rel(tag) # Change THIS
#         label = get_label_ent(tag)
        if label == 'NotFound':
            continue
        scatter = ax.scatter([x1[0], x2[0]], [x1[1], x2[1]], c = color_label_map[label], alpha=1.0, label=label)
        plt.plot([x1[0], x2[0]], [x1[1], x2[1]], color="black", linewidth=0.1)
        legend_partition.append(scatter)
        tooltip = mpld3.plugins.PointHTMLTooltip(scatter, labels=[tooltip_style(tag)])
        mpld3.plugins.connect(fig, tooltip)
        
    print 'Legend: '
    for i, (key, value) in enumerate(color_label_map.items()):
        print str(i) + ' = ' + str(key)
        l_nums.append(str(i))
        l_keys.append(key)
        recs.append(mpatches.Rectangle((0,0),1,1,fc=value))

    plt.legend(recs, l_keys, loc='upper right', fontsize=8)
    if title:
        ax.set_title(title, size=12)
        
    mpld3.save_html(fig, title + '.html')
    mpld3.show()
#     mpld3.display() # doesn't seem to work for ipython notebooks?
    return

In [3]:
trainlidx, trainridx, trainoidx, validlidx, validridx, validoidx, \
testlidx, testridx, testoidx, true_triples, KB = load_FB15k_data()

entity2idx = cPickle.load(open(datapath + dataset + '_entity2idx.pkl'))
idx2entity = dict([(j, i) for (i, j) in entity2idx.items()])
entity2name = cPickle.load(open(datapath + dataset + '_idx2entityname'))


  (15742, 0)	1.0
  (16224, 1)	1.0
  (15797, 2)	1.0
  (15046, 3)	1.0
  (16051, 4)	1.0
  (16119, 5)	1.0
  (15683, 6)	1.0
  (15550, 7)	1.0
  (16049, 8)	1.0
  (15506, 9)	1.0
  (15037, 10)	1.0
  (15028, 11)	1.0
  (15921, 12)	1.0
  (15041, 13)	1.0
  (15919, 14)	1.0
  (15886, 15)	1.0
  (15031, 16)	1.0
  (15545, 17)	1.0
  (15978, 18)	1.0
  (15050, 19)	1.0
  (15546, 20)	1.0
  (15534, 21)	1.0
  (15041, 22)	1.0
  (15860, 23)	1.0
  (15755, 24)	1.0
  :	:
  (15978, 483117)	1.0
  (15917, 483118)	1.0
  (15045, 483119)	1.0
  (15864, 483120)	1.0
  (16039, 483121)	1.0
  (15578, 483122)	1.0
  (15871, 483123)	1.0
  (15167, 483124)	1.0
  (15889, 483125)	1.0
  (15137, 483126)	1.0
  (15049, 483127)	1.0
  (15798, 483128)	1.0
  (15335, 483129)	1.0
  (15576, 483130)	1.0
  (15710, 483131)	1.0
  (15978, 483132)	1.0
  (15545, 483133)	1.0
  (15869, 483134)	1.0
  (16043, 483135)	1.0
  (15049, 483136)	1.0
  (15037, 483137)	1.0
  (15042, 483138)	1.0
  (15051, 483139)	1.0
  (15981, 483140)	1.0
  (15802, 483141)	1.0
(483

In [ ]:
# entity2name, FB15k_ents = {}, set(entity2idx.keys())
# with open(datapath + 'FBmid_to_name.tsv') as f:
#     for line in f:
#         line = line.strip().split('\t')
#         mid, name = line[0], line[1]
#         if mid in FB15k_ents:
#             entity2name[entity2idx[mid]] = name
# print len(entity2name)
# print entity2name
# with open(datapath + dataset + '_idx2entityname', 'w') as f:
#     cPickle.dump(entity2name, f, -1)
# entity2name = cPickle.load(open(datapath + dataset + '_idx2entityname'))
        

In [ ]:
### histogram of test relation occurance
print len(testoidx)
relsids = testoidx - 14951
print len(idx2entity)
# print idx2entity
relnames = [idx2entity[i] for i in testoidx]
# cntr = Counter(relnames)
plt.hist(relsids, bins=1345)
plt.xlabel('relation id')
plt.ylabel('Number of test triples expressing the rel id')
plt.title('Histogram of the Occurance of Relation in FB15k Test Data')
plt.show()
# print cntr

In [ ]:
### histogram of train relation occurance
print len(trainoidx)
relsids = trainoidx - 14951
relnames = [idx2entity[i] for i in trainoidx]
# cntr = Counter(relnames)
plt.hist(relsids, bins=1345)
plt.xlabel('Relation ID')
plt.ylabel('Number of Train Triples Expressing the Relation ID')
plt.title('Histogram of the Occurance of Relation in FB15k Train Data')
plt.show()
# print cntr

In [ ]:
### get categories of relationships
# path = '/Users/corbinrosset/Dropbox/Arora/QA-code/src/KBC_Text/outputs/FB15k_TransE/BEST_TransE_L1_ndim_100_marg_1.5_lrate_0.01_cost_margincost_reg_0.01_REL/'
# entyEmbs, relEmbs = load_embeddings(path + 'best_valid_model.pkl')
# relnames = [str(idx2entity[i+14951]) for i in range(len(relEmbs))]
# words = [i for j in relnames for i in j.strip().split('/') if len(i) > 0]
# print len(words)
# cntr = Counter(words)
# # print cntr.most_common(100)
# print map(lambda (x, y): x, filter(lambda (name, count) : count > 50, cntr.most_common(1000)))

### get categories of entities
# words = [i for j in entity2name.values() for i in j.strip().split()]
# print len(words)
# cntr = Counter(words)
# # print cntr.most_common(100)
# print map(lambda (x, y): x, filter(lambda (name, count) : count > 20, cntr.most_common(1000)))



In [5]:
###  T-SNE plot of entity embeddings from various models
base = '/Users/corbinrosset/Dropbox/Arora/QA-code/src/KBC_Text/outputs/'
# model = 'FB15k_TransE/BEST_TransE_L1_ndim_100_marg_1.5_lrate_0.01_cost_margincost_reg_0.01_REL'
model = 'FB15k_BilinearDiagExtended/BilinearDiagExtended_L1_ndim_200_marg_0.2_lrate_0.01_cost_margincost_pos_high_reg_0.01'
# model = 'FB15k_BilinearDiag/BEST_BilinearDiag_Dot_ndim_100_marg_0.2_lrate_0.01_cost_margincost_pos_high_reg_0.01_REL'
# model = 'FB15k_Bilinear/BEST_Bilinear_Dot_ndim_50_marg_0.2_lrate_0.01_cost_margincost_pos_high_reg_0.01_REL'
# model = 'FB15k_ModelE/BEST_ModelE_Sum_ndim_100_marg_1.0_lrate_0.01_cost_margincost_pos_high_reg_0.01_REL'

entyEmbs, relEmbsL, relEmbsR = load_embeddings(base + model + '/best_valid_model.pkl')
print np.shape(entyEmbs), np.shape(relEmbsL)

####################################### relations
# X_2d = t_sne(relEmbsL) #
# print 'done with tsne ' + str(np.shape(X_2d))
# popup_labels = [str(idx2entity[i+14951]) for i in range(len(relEmbsL))]
# plot_embeddings(X_2d, range(len(relEmbsL)), popup_labels, 'T-SNE plot of Tail Relation Embeddings for ' + model.split('/')[-1])
         
####################################### 2-way relations
# testlidx = testlidx[:1000]
# testoidx = testoidx[:1000]
# testridx = testridx[:1000]
X1 = np.multiply(entyEmbs[testlidx, :], relEmbsL[(testoidx-14951), :])
X2 = np.multiply(entyEmbs[testridx, :], relEmbsR[(testoidx-14951), :])
X2_rand = np.multiply(entyEmbs[testridx, :], relEmbsR[np.random.randint(0, high=1345, size=np.shape(testoidx)), :]) 
######### stop here for a second
X_L1dist = np.linalg.norm(X1 - X2, 1, axis = 1)
X_L1dist_random = np.linalg.norm(X1 - X2_rand, 1, axis = 1)

print np.shape(X_L1dist)
# print X_L1dist
print np.percentile(X_L1dist, range(0, 100, 10))
print np.percentile(X_L1dist_random, range(0, 100, 10))

######## resume here:
# n = np.shape(X1)[0]
# print np.shape(X1), np.shape(X2)
# X = np.vstack([X1, X2])
# popup_labels = [str(idx2entity[i]) for i in testoidx] * 2
# labels = [1 for i in testoidx] + [2 for i in testoidx] # list(testoidx)*2
# assert X.shape[0] == len(popup_labels)

# X_2d = t_sne(X) #
# print 'done with tsne ' + str(np.shape(X_2d))
# plot_embedding_pairs(X_2d[:n, :], X_2d[n:, :], labels, popup_labels, 'T-SNE plot of Tail Relation Embeddings for ' + model.split('/')[-1])
      
    
####################################### entities
# X_2d = t_sne(entyEmbs) #
# print 'done with tsne ' + str(np.shape(X_2d))
# popup_labels = [str(entity2name.get(i, 'unknown')) for i in range(len(entyEmbs))]
# plot_embeddings(X_2d, range(len(entyEmbs)), popup_labels, 'T-SNE plot of Entity Embeddings for ' + model.split('/')[-1])
    
    

loading model from file: /Users/corbinrosset/Dropbox/Arora/QA-code/src/KBC_Text/outputs/FB15k_BilinearDiagExtended/BilinearDiagExtended_L1_ndim_200_marg_0.2_lrate_0.01_cost_margincost_pos_high_reg_0.01/best_valid_model.pkl
(16296, 200) (1345, 200)
(59071,)
[ 0.          0.58295835  0.81580105  1.02205984  1.25977066  1.45958476
  1.71706667  1.93378204  2.2669627   2.49077724]
[ 0.18190672  0.65344461  0.83380231  1.02839852  1.19150549  1.35639841
  1.51422798  1.6795864   1.89328461  2.07296602]
